In [4]:
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

import tensorflow_hub as hub

from tensorflow.keras import layers
import bert

In [32]:
import pandas as pd
import re
import random
import math

# read data CSV

In [9]:
movie_reviews = pd.read_csv("./IMDB_Dataset.csv")
movie_reviews.isnull().values.any()
movie_reviews.shape

(50000, 2)

# preprocessing

In [10]:
def preprocess_text(sen):
    # Removing html tags
    sentence = remove_tags(sen)

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

In [13]:
TAG_RE = re.compile(r'<[^>]+>')
def remove_tags(text):
    return TAG_RE.sub('', text)

# prepare X

In [14]:
reviews = []
sentences = list(movie_reviews['review'])
for sen in sentences:
    reviews.append(preprocess_text(sen))

# prepare Y

In [15]:
movie_reviews.sentiment.unique()

array(['positive', 'negative'], dtype=object)

In [16]:
y = movie_reviews['sentiment']
y = np.array(list(map(lambda x: 1 if x=="positive" else 0, y)))

In [43]:
y[0]

1

# tokenizer

In [18]:
BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

In [21]:
def tokenize_reviews(text_reviews):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text_reviews))

In [20]:
tokenizer.tokenize("don't be so judgmental")

['don', "'", 't', 'be', 'so', 'judgment', '##al']

In [22]:
tokenizer.convert_tokens_to_ids(tokenizer.tokenize("don't be so judgmental"))

[2123, 1005, 1056, 2022, 2061, 8689, 2389]

In [23]:
tokenized_reviews = [tokenize_reviews(review) for review in reviews]

In [24]:
all_data = [(review, y[i])
                 for i, review in enumerate(tokenized_reviews)]

In [85]:
run_all_data = [(review_lab[0], review_lab[1]) for review_lab in reviews_with_len]

In [65]:
all_data[0]

[[2054, 5896, 2054, 2466, 2054, 6752], 0]

In [86]:
processed_dataset = tf.data.Dataset.from_generator(lambda: run_all_data, output_types=(tf.int32, tf.int32))

# create hyperaprameter

In [87]:
BATCH_SIZE = 32
batched_dataset = processed_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))

In [88]:
next(iter(batched_dataset))

(<tf.Tensor: id=23541, shape=(32, 678), dtype=int32, numpy=
 array([[ 2028,  1997,  1996, ...,     0,     0,     0],
        [ 1037,  6919,  2210, ...,     0,     0,     0],
        [ 1045,  2245,  2023, ...,     0,     0,     0],
        ...,
        [ 2162,  3185,  2003, ...,  7874, 27466, 10462],
        [21642,  1998,  7554, ...,     0,     0,     0],
        [12098, 16425,  2938, ...,     0,     0,     0]], dtype=int32)>,
 <tf.Tensor: id=23542, shape=(32,), dtype=int32, numpy=
 array([1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0,
        1, 0, 0, 1, 1, 0, 0, 1, 1, 1], dtype=int32)>)

In [89]:
TOTAL_BATCHES = math.ceil(len(all_data) / BATCH_SIZE)
TEST_BATCHES = TOTAL_BATCHES // 5

batched_dataset.shuffle(TOTAL_BATCHES)
test_data = batched_dataset.take(TEST_BATCHES)
train_data = batched_dataset.skip(TEST_BATCHES)

# create the model 

In [90]:
class TEXT_MODEL(tf.keras.Model):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dimensions=128,
                 cnn_filters=50,
                 dnn_units=512,
                 model_output_classes=2,
                 dropout_rate=0.1,
                 training=False,
                 name="text_model"):
        super(TEXT_MODEL, self).__init__(name=name)
        
        self.embedding = layers.Embedding(vocabulary_size,
                                          embedding_dimensions)
        self.cnn_layer1 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=2,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer2 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=3,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer3 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=4,
                                        padding="valid",
                                        activation="relu")
        self.pool = layers.GlobalMaxPool1D()
        
        self.dense_1 = layers.Dense(units=dnn_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if model_output_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=model_output_classes,
                                           activation="softmax")
    
    def call(self, inputs, training):
        l = self.embedding(inputs)
        l_1 = self.cnn_layer1(l) 
        l_1 = self.pool(l_1) 
        l_2 = self.cnn_layer2(l) 
        l_2 = self.pool(l_2)
        l_3 = self.cnn_layer3(l)
        l_3 = self.pool(l_3) 
        
        concatenated = tf.concat([l_1, l_2, l_3], axis=-1) # (batch_size, 3 * cnn_filters)
        concatenated = self.dense_1(concatenated)
        concatenated = self.dropout(concatenated, training)
        model_output = self.last_dense(concatenated)
        
        return model_output

In [91]:
VOCAB_LENGTH = len(tokenizer.vocab)
EMB_DIM = 200
CNN_FILTERS = 100
DNN_UNITS = 256
OUTPUT_CLASSES = 2

DROPOUT_RATE = 0.2

NB_EPOCHS = 5

In [92]:
text_model = TEXT_MODEL(vocabulary_size=VOCAB_LENGTH,
                        embedding_dimensions=EMB_DIM,
                        cnn_filters=CNN_FILTERS,
                        dnn_units=DNN_UNITS,
                        model_output_classes=OUTPUT_CLASSES,
                        dropout_rate=DROPOUT_RATE)

In [93]:
if OUTPUT_CLASSES == 2:
    text_model.compile(loss="binary_crossentropy",
                       optimizer="adam",
                       metrics=["accuracy"])
else:
    text_model.compile(loss="sparse_categorical_crossentropy",
                       optimizer="adam",
                       metrics=["sparse_categorical_accuracy"])

In [ ]:
text_model.fit(train_data, epochs=NB_EPOCHS)

Epoch 1/5
1251/1251 [==============================] - 939s 751ms/step - loss: 0.3133 - accuracy: 0.8580
Epoch 2/5
1251/1251 [==============================] - 3221s 3s/step - loss: 0.1334 - accuracy: 0.9514
Epoch 3/5
1251/1251 [==============================] - 1052s 841ms/step - loss: 0.0626 - accuracy: 0.9774
Epoch 4/5
 382/1251 [========>.....................] - ETA: 11:58 - loss: 0.0571 - accuracy: 0.9782